In [2]:
import torch
import os
print(os.getcwd())
gene = torch.load("./weights/g_02500000")
disc = torch.load("./weights/do_02500000")

/home/ynishimura/workspace/python/vc_tts_template/vc_tts_template/vocoder/hifigan


In [8]:
print(disc["steps"])
print(disc["epoch"])

2500000
92


In [19]:
import torch
model = torch.nn.Linear(10,10)
scheduler = torch.optim.lr_scheduler.ExponentialLR(
    torch.optim.AdamW(model.parameters(), lr=0.0002), 0.999
)
print(scheduler.state_dict())

for _ in range(92):
    scheduler.step()
print(scheduler.state_dict())

{'gamma': 0.999, 'base_lrs': [0.0002], 'last_epoch': 0, '_step_count': 1, 'verbose': False, '_get_lr_called_within_step': False, '_last_lr': [0.0002]}
{'gamma': 0.999, 'base_lrs': [0.0002], 'last_epoch': 92, '_step_count': 93, 'verbose': False, '_get_lr_called_within_step': False, '_last_lr': [0.00018241263313645436]}


/home/ynishimura/miniconda3/envs/vc_tts_template/lib/python3.8/site-packages/torch/optim/lr_scheduler.py:128: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


In [20]:
state_dict = {
    "state_dict": {
        "netG": gene['generator'],
        "netMPD": disc['mpd'],
        "netMSD": disc['msd'],
    },
    "optimizer_state": {
        "g": disc['optim_g'],
        "d": disc['optim_d'],
    },
    "lr_scheduler_state": {
        "g": scheduler.state_dict(),
        "d": scheduler.state_dict()
    }
}

In [21]:
torch.save(state_dict, "./weights/universal.pth")